In [1]:
from pydantic import BaseModel, Field

class ReviewResults(BaseModel):
    structure_check: str = Field(default="", description="構造")
    content_check: str = Field(default="", description="記事内容")
    sentence_quality_check: str = Field(default="", description="文章の質")


class ReflectionResults(BaseModel):
    judgement: str = Field(default="", description="OK/NG判断")
    feedback: str = Field(default="", description="改善提案")
    retry_count: int = Field(default=0, description="リトライ数")


# State
class State(BaseModel):
    article: str = Field(default="", description="記事内容")
    review_results: ReviewResults = Field(
        default_factory=ReviewResults,
        description="レビュー結果"
    )
    review_summary: str = Field(default="", description="レビュー統合結果")
    reflection_result: ReflectionResults = Field(
        default_factory=ReflectionResults,
        description="内省結果"
    )

In [4]:
from langgraph.graph import StateGraph
workflow = StateGraph(State)

In [ ]:
from typing import Any
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv('../../.env')
import os
openai_api_key = os.getenv('OPENAI_API_KEY')


from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


def content_check(state: State) -> dict[str, Any]:
    article = state.article
    llm = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
    

    prompt = PromptTemplate(
        input_variables=["article"],

        template="""
        あなたはプロフェッショナルな技術ブログの校正・校閲者です。
        あなたのタスクは記事内容について、以下の5つの観点に基づき、「評価ポイント」と「改善点」を出力する ことです。

        # レビュー観点
        1. **技術的正確性**
        - 専門用語の誤用がないか。

        2. **独創性・独自性**
        - 具体的な事例が含まれているか。
        - 他の記事との差別化ができているか。

        3. **網羅性**
        - 他に加えるべき観点がないか。

        4. **内容の質**
        - 一般的な情報のなぞりになっていないか。
        - 内容が簡単すぎないか。

        5. **倫理性・適切性**
        - 誤解や不快感を与える表現がないか。
        - 企業の機密情報やブランドを損なう内容がないか。

        # 留意事項
        - 「改善点」は具体的に記述してください
        - 改善点がない場合は「改善点」の欄に「なし」と明記してください
        - 文章は「です・ます」調で統一してください

        # 記事内容
        {article}

        # 出力フォーマット        
        **観点1: 技術的正確性**
        - **評価ポイント**: {{良い点を一文で記載}}
        - **改善点**:
        - {{改善点がある場合は記載し、ない場合は「なし」と記載}}

        **観点2: 独創性・独自性**
        - **評価ポイント**: {{良い点を一文で記載}}
        - **改善点**:
        - {{改善点がある場合は記載し、ない場合は「なし」と記載}}

        **観点3: 網羅性**
        - **評価ポイント**: {{良い点を一文で記載}}
        - **改善点**:
        - {{改善点がある場合は記載し、ない場合は「なし」と記載}}

        **観点4: 内容の質**
        - **評価ポイント**: {{良い点を一文で記載}}
        - **改善点**:
        - {{改善点がある場合は記載し、ない場合は「なし」と記載}}

        **観点5: 倫理性・適切性**
        - **評価ポイント**: {{良い点を一文で記載}}
        - **改善点**:
        - {{改善点がある場合は記載し、ない場合は「なし」と記載}}
    """,
    )
    parser = StrOutputParser()
    chain = prompt | llm | parser

    # 実行して結果を取得
    result = chain.invoke({"article": article})
    state.review_results.content_check = result

    return state